In [24]:
from time import time
import numpy as np
import mem_sdrsdm as m
from mem_sdrsdm2 import DiadicMemory_Orig, DiadicMemory_Counters, DiadicMemory_SdmCounters, DiadicMemory_SdmCountersAndRetrieval
from sdrsdm import DiadicMemory
from sdr_util import random_sdrs
from tqdm.notebook import tqdm
import time
import pandas as pd

In [8]:
SDR_SIZE = 1000
SDR_BITS = 10
XCOUNT = 10000

In [9]:
# SDRs -> binary arrays
def expand_sdrs_ndarray(sdrs, sdr_size):
    z = np.zeros((sdrs.shape[0], sdr_size), dtype=int).reshape(-1)
    non_zero_indices = (sdrs.T + np.arange(0, sdr_size * sdrs.shape[0], sdr_size)).T.reshape(-1)
    z[non_zero_indices] = 1
    return z.reshape((sdrs.shape[0], sdr_size))

def expand_sdrs_list(sdrs, sdr_size):
    subsdrs = []
    rv = None

    for sdr in sdrs:
        if not subsdrs or len(sdr) == len(subsdrs[0]):
            subsdrs.append(sdr)
            continue
        
        b = expand_sdrs_ndarray(np.array(subsdrs), sdr_size)
        rv = b if rv is None else np.concat([rv, b])
        subsdrs = [sdr]

    if subsdrs:
        b = expand_sdrs_ndarray(np.array(subsdrs), sdr_size)
        rv = b if rv is None else np.concat([rv, b])

    return rv

# binary arrays-> SDRs
def compact_binaries(bins, sdr_size):
    return np.argwhere(bins > 0)[:,1].reshape(bins.shape[0],-1)

def hamming_dist(bins1, bins2):
    return np.count_nonzero(bins1 != bins2, axis=1)

In [10]:
a1 = np.array(random_sdrs(2, sdr_size=SDR_SIZE, on_bits=SDR_BITS))
a2 = np.array(random_sdrs(2, sdr_size=SDR_SIZE, on_bits=SDR_BITS))
a1, a2

(array([[ 23,  31,  57,  65,  91, 144, 553, 691, 803, 813],
        [ 70, 174, 202, 291, 345, 510, 599, 698, 770, 910]], dtype=uint32),
 array([[ 34, 155, 283, 405, 453, 455, 475, 506, 751, 921],
        [167, 521, 533, 677, 682, 683, 704, 705, 781, 794]], dtype=uint32))

In [11]:
z1 = expand_sdrs_ndarray(a1, SDR_SIZE)
z2 = expand_sdrs_ndarray(a2, SDR_SIZE)
assert np.array_equal(compact_binaries(z1, SDR_SIZE), a1)
assert np.array_equal(compact_binaries(z2, SDR_SIZE), a2)
assert np.all(hamming_dist(z1, z1) == 0)
assert np.all(hamming_dist(z2, z2) == 0)
assert np.all(hamming_dist(z1, z2) > 0)

In [12]:
b1 = random_sdrs(2, sdr_size=SDR_SIZE, on_bits=SDR_BITS)
b2 = random_sdrs(2, sdr_size=SDR_SIZE, on_bits=SDR_BITS + 1)
expand_sdrs_list(b1 + b2, SDR_SIZE);

In [101]:
mem_types = [DiadicMemory_Orig, DiadicMemory_Counters, DiadicMemory_SdmCounters, DiadicMemory_SdmCountersAndRetrieval]
results = pd.DataFrame([], columns=['Type', 'Duration', 'Hamm. dist'])

for i, mem_type in tqdm(enumerate(mem_types), total=len(mem_types)):
    start_ts = time.time()
    
    mem = mem_type(SDR_SIZE, SDR_BITS)
    x = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)
    y = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)
    
    for x_, y_ in zip(x, y): mem.store(x_, y_)
    
    y2 = [mem.query(x_) for x_ in x]
    yb = expand_sdrs_ndarray(np.array(y), SDR_SIZE)
    y2b = expand_sdrs_list(y2, SDR_SIZE)
    hds = hamming_dist(yb, y2b)
    
    end_ts = time.time()
    duration = (end_ts - start_ts)

    for hd_bin in np.array(np.unique_counts(hds)).T:
        results.loc[len(results)] = [f'{i} {mem_type.__name__}', duration, hd_bin]


  0%|          | 0/4 [00:00<?, ?it/s]

,Type,Duration,Hamm. dist
0,0 DiadicMemory_Orig,3.509741,"[0, 10000]"
1,1 DiadicMemory_Counters,1.661758,"[0, 10000]"
2,2 DiadicMemory_SdmCounters,7.074571,"[0, 9533]"
3,2 DiadicMemory_SdmCounters,7.074571,"[992, 2]"
4,2 DiadicMemory_SdmCounters,7.074571,"[993, 15]"
5,2 DiadicMemory_SdmCounters,7.074571,"[994, 30]"
6,2 DiadicMemory_SdmCounters,7.074571,"[995, 89]"
7,2 DiadicMemory_SdmCounters,7.074571,"[996, 99]"
8,2 DiadicMemory_SdmCounters,7.074571,"[997, 106]"
9,2 DiadicMemory_SdmCounters,7.074571,"[998, 87]"


In [9]:
mem = MemDiadicMemory(SDR_SIZE, SDR_BITS)
x = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)
y = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)

for x_, y_ in zip(x, y): mem.store(x_, y_)

y2 = [mem.query(x_) for x_ in x]

yb = expand_sdrs_ndarray(np.array(y), SDR_SIZE)
y2b = expand_sdrs_list(y2, SDR_SIZE)

hds = hamming_dist(yb, y2b)
np.array(np.unique_counts(hds))

array([[    0],
       [10000]])

In [10]:
mem = MemDiadicMemory2(SDR_SIZE, SDR_BITS)
x = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)
y = random_sdrs(XCOUNT, sdr_size=mem.N, on_bits=mem.P)

for x_, y_ in zip(x, y): mem.store2(x_, y_)

y2 = [mem.query2(x_) for x_ in x]

yb = expand_sdrs_ndarray(np.array(y), SDR_SIZE)
y2b = expand_sdrs_list(y2, SDR_SIZE)

hds = hamming_dist(yb, y2b)
np.array(np.unique_counts(hds))

array([[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10],
       [7335,   39,   83,  133,  119,  135,  159,  213,  243,  410, 1131]])